In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random

random.shuffle(data)

print(data)

In [ ]:
import re
st = data[2]['content']
st=re.sub("\n","",st)
st=re.sub("\\\.*?\s","",st)
print(st)

In [2]:
import json

with open('data.json', 'r',encoding='utf-8') as f:
    data = json.load(f)
    data=data['response']['docs']
len(data)

100000

In [6]:
train=list()
test=list()
for i in range(0,11):
    st = data[i]['content']
    st=re.sub("\n","",st)
    st=re.sub("\\\.*?\s","",st)
    train.append(st)
for i in range(11,18):
    st = data[i]['content']
    st=re.sub("\n","",st)
    st=re.sub("\\\.*?\s","",st)
    test.append(st)

In [2]:
from fetch_data import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models

import sys
sys.path.append('..')
from utils import preprocess, get_windows

F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
MIN_COUNTS = 20
MAX_COUNTS = 1800
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 15
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 5
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [4]:
nlp = spacy.load('en')

# Load dataset

In [5]:
from fetch_data import fetch_20newsgroups
dataset = fetch_20newsgroups(data_home='D:/code python/LDA2vec/bangla news',subset='all', remove=('headers', 'footers', 'quotes'))
docs = dataset['data']

In [6]:
# number of documents
len(docs)

99990

In [7]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

In [8]:
print(docs[0])

(0, 'রাঙ্গাবালী উপজেলাText size A A AColor C C C Cপাতাউপজেলার পটভূমি১। অবস্থানঃ পটুয়াখালী জেলার সর্বদক্ষিনে বঙ্গোপসাগরের পাদদেশে\xa0 উত্তরে চালিতাবুনিয়া নদী,\xa0 আগুনমূখা নদী ও চর বিশ্বাস, পশ্চিমে রামনাবাদ চ্যানেল ও কলাপাড়া উপজেলা পূর্বে চর ফ্যাশন\xa0 উপজেলার চর কুররী- মুকরী\xa0 এবং দক্ষিনে বঙ্গোপসাগর। \xa0২। উপজেলার নামকরনঃ রাঙ্গাবালী উপজেলার নামকরনের সঠিক ইতহাস জানা যায়নি। \xa0তবে কথিত আছে যে, সাগর বক্ষে নতুন বালুচর সৃষ্টির ফলে কালের বিবর্তনে এই বালুচরের বালু লাল\xa0 ছিল। \xa0এই\xa0 ‘‘লাল’’ \xa0শব্দটি আঞ্চলিক ভাষায় রাঙ্গা নামে পরিচিত। এ থেকে রাঙ্গাবালি নামের উৎপত্তি। ইতিহাসবেত্তাগন জানান ১৭৮৪ সালে কতিপয় রাকাইন জনগোষ্ঠী আরাকান রাজ্য থেকে পালিয়ে এসে এ অঞ্চলে বসতি স্থাপন করে। তখন থেকে এতদঞ্চলে \xa0জনবসতি শুরু হয়।৩। উপজেলার উৎপত্তিঃ ৭ জুন ২০১১ তারিখে নিকারের (ন্যাশনাল ইপপ্লিমেন্টেশন কমিটি ফর এ্যাডমিনিস্ট্রেশন রিফর্ম )\xa0 ১০৫তম সভায় রাঙ্গাবালী উপজেলার প্রশাসনিক\xa0 অনুমোদন হয়। এরই ধারাবাহিকতায় গত ১৩ জুন ২০১১ তারিখে বাংলাদেশ গেজেট প্রকাশিত হয়।')


# Preprocess dataset and create windows

In [9]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

  0%|                                                                             | 61/99990 [00:09<4:28:03,  6.21it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████████████████████████████████████████████████████████████████████| 99990/99990 [3:34:43<00:00,  7.76it/s]


number of removed short documents: 33312
total number of tokens: 9784186
number of tokens to be removed: 4987079
number of additionally removed short documents: 21326
total number of tokens: 4611025

minimum word count number: 1
this number can be less than MIN_COUNTS because of document removal


In [10]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [11]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

12026it [00:09, 1221.67it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
45352it [00:32, 1396.10it/s]


In [12]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

12

In [13]:
# number of windows (equals to the total number of tokens)
data.shape[0]

4611025

# Get unigram distribution

In [14]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [15]:
%%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=5, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
    word_vectors[i] = model.wv[str(i)]

KeyError: "word '32463' not in vocabulary"

In [16]:
# number of unique words
vocab_size

32589

# Prepare initialization for document weights

In [17]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [43]:
%%time
n_topics = 40
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

Wall time: 1min 30s


In [36]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : প্রেমিকা প্রেমপত্র অনুভব অনুভুতি অনুভূতি আক্রান্ত শিশুর হাত ভাল কারনে
topic 1 : ged consultative forestry smes broadly seeking much psd demographer ipid
topic 2 : civics(269,270 c(267,268 সম্প্রসারণ অবহিত থানার ভারপ্রাপ্ত গ্রেফতার অপরাধ parishad পরিষদকে
topic 3 : রেজিস্ট্রেশন দরপত্র ভাতা বিয়ে কর্মকর্তার প্রেরণ প্রাপ্তির প্রাপ্ত অনুসারে সরবরাহ
topic 4 : খতিয়ান বৈশাখ রেকর্ড মালিক প্রস্তুত ক্রমিক সম্পত্তি দাগ নাম্বার প্রত্যেকটি
topic 5 : woo মামলা মামলার আদালতের আদালতে 35.00 maria xiaomi বিচার sale
topic 6 : বিনিয়োগ অর্থনৈতিক খাতে দশমিক পোশাক দাম গবেষণা ব্যাংকের পাশাপাশি ডলার
topic 7 : results calendar login attendance lab mission administration club panel portal
topic 8 : wamy construction font tender document click procurement windows file apartment
topic 9 : core intel gen তেল খাবার optiplex 6th review(s পদ্ধতি series
topic 10 : বিদ্যালয়টি ভবন কলেজের শিক্ষার্থীদের লাভ প্রতিষ্ঠিত স্কুলের পরীক্ষায় বিজ্ঞান অধ্যাপক
topic 11 : usb music device charge output battery wireless sound

In [20]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : login results mission lab calendar club vision english panel facilities
topic 1 : প্রেমিকা প্রেমপত্র বৈশাখ অনুভব অনুভুতি অনুভূতি রেড সুখ ভাষা বুঝবে
topic 2 : dye yarn chemical easy fiber device dyeing range charge oil
topic 3 : public_html বিরোধী /home জঙ্গী সমাবেশসন্ত্রাস core সম্প্রসারণ intel plugin counter
topic 4 : ক্যালেন্ডার কোডঃ বৃত্তি ক্লাশ পাঠ্যক্রম কার্টে ব্যয় নবায়ন পাঠ্যতালিকা ফটো
topic 5 : ভাতা mba সমাজসেবা dolore কেন্দ্রে কল্যাণ রোগীদের আগত দেয়া qui
topic 6 : 1,100.00 850.00 woo 3627 999.00 আদালতের logo 2,250.00 20.00 35.00
topic 7 : রেজিস্ট্রেশন বিয়ে বিয়ের মৃত্যু দলিল বিবাহ অফিসে নামজারি খাস কপি
topic 8 : rise gdp poor domestic revenue loan ldc however progress estimate
topic 9 : বিনিয়োগ খাতে অর্থনৈতিক দশমিক পোশাক দাম ব্যাংকের রাজস্ব গবেষণা ডলার
topic 10 : রাসত্মা নির্মান পর্যমত্ম পাকা গভীর নলকুপ পশ্চিম পাশে মৃধা সংলগ্ন
topic 11 : রেজাল্ট লিঙ্ক শীর্ষক বিজ্ঞপ্তি ডায়নামিক অনুরোধ corner খ্রি শিক্ষাবর্ষে ধন্যবাদ
topic 12 : 4th vol hasan mohammad m.a. 2009 japan 

In [40]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : আওয়ামী nunc সম্পাদক aliquam libero euismod nibh ante erat sagittis
topic 1 : রেজিস্ট্রেশন বিয়ে কর্মকর্তার নবায়ন বিয়ের অফিসের দলিল কপি কোড বিবাহ
topic 2 : শীর্ষক ডায়নামিক অনুরোধ ধন্যবাদ কক্ষে bdt খ্রি শিক্ষাবর্ষে থাকার নির্দেশ
topic 3 : dye yarn chemical oil fiber dyeing method range fibre delivery
topic 4 : login tender administration cda links portal printing publications principal attendance
topic 5 : খতিয়ান মালিক প্রস্তুত দরপত্র সমাজসেবা সম্পত্তি ভাতা সভায় রেকর্ড মোতাবেক
topic 6 : party domestic gdp rise loan revenue big poor ldc progress
topic 7 : সম্প্রসারণ মৃত্যু বরিশাল সময় মুসলিম প্রযুক্তি আল্লাহ কৃষকদের সার উৎপাদন
topic 8 : রাসত্মা নির্মান পর্যমত্ম পশ্চিম গভীর পাকা নলকুপ পাশে পূর্ব মৃধা
topic 9 : fringilla bibendum তেল ãƒâ€šã‚â ãƒæ’ã¢â‚¬å¡ãƒâ€šã‚â admin ali vulputate semper cras
topic 10 : আক্রান্ত খাবার শিশুর core intel সাহায্য রোগ থাকলে কারনে ভাল
topic 11 : host উইকেট server client manage ম্যাচের software দ্বিতীয় ভারতীয় personal
topic 12 : 1,100.00 850.00 3627 99

In [28]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : বিদ্যালয়টি শিক্ষার্থীদের ভবন অবস্থিত কলেজের বিজ্ঞান লাভ পরীক্ষায় প্রতিষ্ঠিত বিদ্যালয়ে
topic 1 : ged consultative forestry smes broadly seeking much psd demographer ipid
topic 2 : woo select 35.00 maria sale parishad welcome rat ইন্টারনেট logo
topic 3 : lab calendar club login results wamy panel facilities mission sports
topic 4 : মিয়া দক্ষিণ আব্দুল চন্দ্র আবুল আলম কুমার মোহাম্মদ আক্তার কিলোমিটার
topic 5 : পরিদর্শক বালিকা bdt ওয়েবসাইটের থাকায় ডেভেলপমেন্ট রোড মাসিক আছেন আন্তরিকভাবে
topic 6 : /home public_html mba counter plugin mechanic buyer statsmechanic.php kashsedu rana
topic 7 : music core xiaomi usb charge intel wireless gen device battery
topic 8 : আক্রান্ত খাবার সাহায্য শিশুর রোগ কারনে নিয়মিত তেল রাখতে থাকলে
topic 9 : বিজ্ঞপ্তি write card certificate payment password ltd. court achievement candidate
topic 10 : dolore qui enim labore voluptatem সমবায় adipisicing s velit magna
topic 11 : রেড জাতীয় আন্দোলনের জরুরী অ্যান্ড শাখা নবায়ন ক্রস ফেডারেশন পুরস্কার
topic 12 : রেজাল

In [44]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : child mr. family house rana plaza want prof bgmea buy
topic 1 : ged consultative forestry smes broadly seeking much psd demographer ipid
topic 2 : নামঃ বিক্রি দাম চাল দক্ষিণ কেজি কর্মসূচী জুন লক্ষ্য সঞ্চয়পত্র
topic 3 : শিক্ষার্থীদের বিজ্ঞান corner attendance portal certificate পরীক্ষায় achievement managing code
topic 4 : বিরোধী civics(269,270 c(267,268 জঙ্গী সমাবেশসন্ত্রাস ãƒâ€šã‚â bdt ãƒæ’ã¢â‚¬å¡ãƒâ€šã‚â general.xls log_2015
topic 5 : dye yarn clothing dyeing jute chemical denim manufacturer strictly purpose
topic 6 : সম্প্রসারণ কর্মকর্তার মহান কল্যাণ কৃষকদের আছেন কমিশনার যোগ্যতা কপি ইংরেজিতে
topic 7 : রাসত্মা পর্যমত্ম নির্মান গভীর নলকুপ পাশে পশ্চিম পূর্ব কালভার্ট পাকা
topic 8 : party poor gdp domestic revenue rise however loan estimate suggest
topic 9 : pro host software output hmi input server network tool interface
topic 10 : খতিয়ান মালিক রেকর্ড প্রস্তুত সম্পত্তি ক্রমিক প্রত্যেকটি নাম্বার দাগ মৌজা
topic 11 : session english ldc club debate publication former vol sports la

In [32]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : রেজিস্ট্রেশন বিয়ে কেন্দ্রে সরবরাহ মুসলিম বিবাহ রোগীদের আল্লাহ আগত দেয়া
topic 1 : আক্রান্ত খাবার শিশুর রোগ ওজন কারনে ক্যান্সার শরীরের ডাক্তার নিন
topic 2 : কার্টে vii ট্যাক্স অন্তর্ভুক্ত viii iii 12:26 চেক যুক্ত theme
topic 3 : party want child family alliance rana have plaza election union
topic 4 : কোডঃ জেলাঃ জেলা- liver উপজেলাঃ পোস্ট ঈদগাহ উপজেলা- নং- থানা
topic 5 : আওয়ামী ডেস্ক দলের story সম্পাদক লীগের নেতা গতকাল চাঁদনী ঘটনা
topic 6 : উন্নয়নের স্থানে খাদ্য উন্নয়ন সভার চাল পাট নিষিদ্ধ প্রয়োজনীয় শস্য
topic 7 : 4th c(267,268 civics(269,270 rent room electricity tag connection apartment bed
topic 8 : বিদ্যালয়টি অবস্থিত ভবন প্রতিষ্ঠিত মাদ্রাসা পূর্ব সনে কক্ষ বিদ্যালয়ে নির্মাণ
topic 9 : ব্যবহারিক শিক্ষার্থীদের স্কুলের শ্রেণির কোর্স সম্পূর্ণ পরীক্ষায় ২০১৫ arabic থাকায়
topic 10 : বালিকা মাসিক পরিদর্শক আছেন ওয়েবসাইটের থাকায় ডেভেলপমেন্ট সবাইকে সাপ্তাহিক স্বাগতম
topic 11 : pro software construction hmi art interface dummy print series designer
topic 12 : 1,100.00 850.00 3627 999

In [24]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : বিদ্যালয়টি ভবন অবস্থিত প্রতিষ্ঠিত লাভ সনে কক্ষ বিদ্যালয়ে পরিবেশে মরহুম
topic 1 : 4359 5517 4190 5049 1,100.00 4188 google pricing images 1,300.00
topic 2 : আওয়ামী সম্পাদক story লীগের দলের গতকাল পুলিশের বিএনপি বিএনপির মামলা
topic 3 : খতিয়ান রেকর্ড মালিক প্রস্তুত সম্পত্তি ক্রমিক প্রত্যেকটি মৌজা দাগ পর্চা
topic 4 : public_html /home plugin counter mechanic kashsedu statsmechanic.php mysql_query অত্যন্ত ভূমিকা
topic 5 : 4th faculty language award mr. vol organization seminar bengali certificate
topic 6 : vii viii চুল ক্যান্সার ভিটামিন ত্বকের iii নিয়মিত former মাত্রা
topic 7 : woo ক্যালেন্ডার বৃত্তি পাঠ্যক্রম ক্লাশ ব্যয় bibendum fringilla পাঠ্যতালিকা 35.00
topic 8 : তেল ভারতীয় দ্বিতীয় কলাপাড়া ম্যাচের মডেল মুস্তাফিজুর মোসাঃ তেলের পাম
topic 9 : child rice agricultural household disease medicine flood care index family
topic 10 : write card court bar code candidate 12:26 send admit council
topic 11 : দরপত্র বিল ইজারা মৃত্যু প্রেরণ প্রাপ্ত স্থানে লাইসেন্স ধারা প্রাপ্তির
topic 12 : বি

In [45]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

 13%|█████████▊                                                                  | 5869/45352 [00:09<01:07, 587.81it/s]F:\InstallationFiles\Anaconda3\envs\for_pytorch\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|███████████████████████████████████████████████████████████████████████████| 45352/45352 [00:46<00:00, 966.08it/s]


# Save data

In [30]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)

In [46]:
np.save('doc_weights_init.npy', doc_weights_init)